<a href="https://colab.research.google.com/github/amirabbasgashtil/Linear-Programming/blob/main/simplex_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplex algorithm for optimizing a characteristic function

# first we have to input characteristic function and constraints

In [53]:
z = []
z =[int(item) for item in input('Enter coeficiant of characteristic function : ').split()]
print(z)

Enter coeficiant of characteristic function : 1 2 3
[1, 2, 3]


In [54]:
m = input('Enter the type of problem: (min or max) : ')

Enter the type of problem: (min or max) : min


In [55]:
number_of_constraints = int(input('Enter the number of constraints: '))

Enter the number of constraints: 2


In [56]:
A = [[int(input('Enter coeficiant of x{} in constraint{} : '.format(i+1, c+1))) for i in range(len(z))] for c in range(number_of_constraints)]

Enter coeficiant of x1 in constraint1 : 1
Enter coeficiant of x2 in constraint1 : 3
Enter coeficiant of x3 in constraint1 : 3
Enter coeficiant of x1 in constraint2 : 2
Enter coeficiant of x2 in constraint2 : 4
Enter coeficiant of x3 in constraint2 : 2


In [57]:
Draws = [input('Enter the Draw of constraint{} : '.format(i+1)) for i in range(number_of_constraints)]

Enter the Draw of constraint1 : =
Enter the Draw of constraint2 : >=


In [58]:
RHS = [input('Enter Right Hand Side in constraint{} : '.format(i+1)) for i in range(number_of_constraints)]

Enter Right Hand Side in constraint1 : 10
Enter Right Hand Side in constraint2 : 12


In [59]:
sign_of_variables = [input('Enter the sign of x{} : (>= / <= / free) '.format(i+1)) for i in range(len(z))]

Enter the sign of x1 : (>= / <= / free) >=
Enter the sign of x2 : (>= / <= / free) <=
Enter the sign of x3 : (>= / <= / free) free


# Now lets convert form of problem to its standard form

1. convert all of constraints to =
2. for all x in characteristic function =>  x>=0

1)

In [60]:
for i in range(len(Draws)):

  if Draws[i] == ">=":
    # we have to add a (-1 * x_n+1) in that constraint and in characteristic function with 0 coeficiant
    z.append(0)
    for j in range(number_of_constraints):
      if i == j:
        A[j].append(-1)
      else:
        A[j].append(0)

  elif Draws[i] == "<=":
    # we have to add a (1 * x_n+1) in that constraint and in characteristic function with 0 coeficiant
    z.append(0)
    for j in range(number_of_constraints):
      if i == j:
        A[i].append(1)
      else:
        A[i].append(0)

print(A)
print(z)

[[1, 3, 3, 0], [2, 4, 2, -1]]
[1, 2, 3, 0]


2)

In [61]:
for i in range(len(sign_of_variables)):
  if sign_of_variables[i] == 'free':
    z.insert(i+1, -1 * z[i])
    for j in range(number_of_constraints):
        A[j].insert(i+1, -1 * A[j][i])
  elif sign_of_variables[i] == '<=':
    z[i] = -1 * z[i]
    for j in range(number_of_constraints):
      A[j][i] = -1 * A[j][i]

print(z)
print(A)

[1, -2, 3, -3, 0]
[[1, -3, 3, -3, 0], [2, -4, 2, -2, -1]]


In [64]:
'''
    now we have an standard optimization problem
    lets print the standard form and then solve it by simplex algorithm
'''
print(m, end="  ")
print('z = ', end="")
for i in range(len(z)):
  print(str(z[i]) + 'x.{}'.format(i+1), end=" ")
print()
print('subject to : ')
for i in range(number_of_constraints):
  for j in range(len(z)):
    print(str(A[i][j]) + '.x{}'.format(j+1), end=" ")
  print('=', end=" ")
  print(RHS[i])


min  z = 1x.1 -2x.2 3x.3 -3x.4 0x.5 
subject to : 
1.x1 -3.x2 3.x3 -3.x4 0.x5 = 10
2.x1 -4.x2 2.x3 -2.x4 -1.x5 = 12
